In [32]:
import asyncssh
import textfsm
import pandas as pd

In [44]:
class Device(asyncssh.client.SSHClient):
    def __init__(self, site_name, ipaddr, username, password):
        self.site_name = site_name
        self.ipaddr = ipaddr
        self.username = username
        self.password = password
        self.jump_connection = None
        self.connection = None
        self.hostname = None
        self.serial_numbers = None
        self.uptime = None
        self.cdp_neighbour_information = None
        self.device_information = None
        self.encryption_algs_list = ["aes128-cbc", "3des-cbc", "aes192-cbc", "aes256-cbc", "aes256-ctr"]
        self.kex_algs_list = ["diffie-hellman-group-exchange-sha1", "diffie-hellman-group14-sha1",
                              "diffie-hellman-group1-sha1"]

    async def connect(self):
        self.connection = \
            await asyncssh.connect(
                self.ipaddr,
                username=self.username,
                password=self.password,
                known_hosts=None,
                encryption_algs=self.encryption_algs_list,
                kex_algs=self.kex_algs_list,
                connect_timeout=10,
            )

    async def initialise(self):
        print("Initialising Device, Please Wait!")
        print(" ")
        await self.get_device_info()
        await self.get_cdp_neighbors()

    async def get_cdp_neighbors(self):
        await self.connect()
        show_cdp_neighbours = await self.connection.run('show cdp neighbors detail')
        with open(f"ProgramFiles/textfsm/cisco_ios_show_cdp_neighbors_detail.textfsm") as f:
            re_table = textfsm.TextFSM(f)
            output = re_table.ParseText(show_cdp_neighbours.stdout)
        self.cdp_neighbour_information = [dict(zip(re_table.header, entry)) for entry in output]

        for entry in self.cdp_neighbour_information:
            text = entry['DESTINATION_HOST']
            head, sep, tail = text.partition('.')
            entry['DESTINATION_HOST'] = head.upper()
            entry['LOCAL_HOST'] = self.hostname
            entry['LOCAL_IP'] = self.ipaddr 
            entry['LOCAL_SERIAL'] = self.serial_numbers
            entry['LOCAL_UPTIME'] = self.uptime
        await self.close()

        return self.cdp_neighbour_information[0]

    async def get_device_info(self):
        await self.connect()
        show_version = await self.connection.run('show version')
        with open(f"ProgramFiles/textfsm/cisco_ios_show_version.textfsm") as f:
            re_table = textfsm.TextFSM(f)
            output = re_table.ParseText(show_version.stdout)
        self.device_information = [dict(zip(re_table.header, entry)) for entry in output]

        self.hostname = self.device_information[0].get("HOSTNAME")
        self.serial_numbers = self.device_information[0].get("SERIAL")
        self.uptime = self.device_information[0].get("UPTIME")
        await self.close()

        return self.device_information[0]

    async def close(self):
        self.connection.close()
        await self.connection.wait_closed()

In [45]:
async def main():
    seed_device = Device('', '', '', '')


    await seed_device.initialise()

    for entry in seed_device.cdp_neighbour_information:
        print(f"Hostname: {entry['DESTINATION_HOST']}")
        print(f"IP Address: {entry['MANAGEMENT_IP']}")
        print(f"Local Port: {entry['LOCAL_PORT']}")
        print(f"Platform: {entry['PLATFORM']}")
        print("=========================================================================")
        
    df = pd.DataFrame(seed_device.cdp_neighbour_information, columns=["LOCAL_HOST",
                                                                      "LOCAL_IP",
                                                                      "LOCAL_PORT",
                                                                      "LOCAL_SERIAL",
                                                                      "LOCAL_UPTIME",
                                                                      "DESTINATION_HOST",
                                                                      "REMOTE_PORT",
                                                                      "MANAGEMENT_IP",
                                                                      "PLATFORM",
                                                                      "SOFTWARE_VERSION",
                                                                      "CAPABILITIES"
                                                                      ])
    print(df)


In [46]:
await main()

Initialising Device, Please Wait!
 
Hostname: OFFICE
IP Address: 192.168.1.3
Local Port: GigabitEthernet0/7
Platform: cisco WS-C3560CG-8PC-S
Hostname: OFFICE
IP Address: 192.168.1.3
Local Port: GigabitEthernet0/8
Platform: cisco WS-C3560CG-8PC-S
Hostname: WM-RTR-C1117
IP Address: 192.168.1.1
Local Port: GigabitEthernet0/1
Platform: cisco C1117-4P
Hostname: UPSTAIRS
IP Address: 192.168.1.4
Local Port: GigabitEthernet0/9
Platform: cisco WS-C2960X-24PS-L
Hostname: UPSTAIRS
IP Address: 192.168.1.4
Local Port: GigabitEthernet0/10
Platform: cisco WS-C2960X-24PS-L
   LOCAL_HOST     LOCAL_IP           LOCAL_PORT   LOCAL_SERIAL  \
0  Agg_Switch  192.168.1.2   GigabitEthernet0/7  [FOC1938Z1BN]   
1  Agg_Switch  192.168.1.2   GigabitEthernet0/8  [FOC1938Z1BN]   
2  Agg_Switch  192.168.1.2   GigabitEthernet0/1  [FOC1938Z1BN]   
3  Agg_Switch  192.168.1.2   GigabitEthernet0/9  [FOC1938Z1BN]   
4  Agg_Switch  192.168.1.2  GigabitEthernet0/10  [FOC1938Z1BN]   

                           LOCAL_UPTIME